In [ ]:
!pip install arch
!pip install yfinance
!pip install mgarch
!pip install qpsolvers
!pip install get_quote_yahoo
!pip install --upgrade pandas-datareader
!pip install --upgrade pandas
!pip install quadprog
!pip install --upgrade scipy==1.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.3/907.3 KB 11.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1

In [ ]:
#Importing packages
import numpy as np
import pandas as pd
#Using the yahoo finance api to grab data
import yfinance as yf
import qpsolvers as qp #Quadratic Programming package
from qpsolvers import solve_qp
from pandas_datareader import data #To get marketcap data from yahoo and other data points
from numpy.linalg import multi_dot #To multiply more than 2 matrices
from sklearn.preprocessing import StandardScaler #To scale data
import arch #to use Garch(1,1)
import matplotlib.pyplot as plt
from pandas_datareader import data
import mgarch

In [ ]:
Data = pd.read_excel('ESG Data(Cleaned).xlsx').set_index('Dates') #Data gathered from Bloomberg and setting index as Dates
Data = Data.drop_duplicates() #Dropping duplicate rows(i.e. dropping holidays and other days that were not actual trading days)
Returns = [] #Initializing returns vector
Dates = [] #Initialzing Dates array
DatesTraded = [] #The dates the actual trade took place
Weights = [] # The weights for each stock ticker on each trade
Capital = 100000 #Amount of capital that is being invested


#Note to self: Start at 4434 for full testing
for i in range(5738,5780,21):#Starting by using approximately 76% of the data to train the model and then increasing
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1] #One period to test the return(i.e. one day return)

#Splitting the S&P500 in the training set
  Benchmark_Training = Training_Data['SPY']

#Splitting the stock data between training and test sets to only include stock price data without the benchmark
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1)
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1)
  m = len(Stocks_Training) #number of observations in the training data
  N = len(Stocks_Training.columns) #Number of stocks

#Returns on training data in the last 21 trading days to be used for Black-Litterman views
  rts_views = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[m-22])/Stocks_Training.to_numpy()[m-22]

#Choosing our views by getting daily averages of historical returns as our views.
  v1 = (rts_views[0]-rts_views[1])/21 #Stock 1 will outperform Stock 2 by v1
  v2 = (rts_views[1]-rts_views[2])/21
  v3 = (rts_views[2]-rts_views[3])/21
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

#Daily returns as (S_i+1 - S_i)/S_i
  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values

#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  ndays = 1 #How many days in the future do you want to estimate the covariance
  cov_nextday = vol.predict(ndays)
  cov = cov_nextday['cov']
  Delta = 1/(2*np.std(rts_Benchmark)) # Delta is equal to "A" in the paper
  hist_avg = np.mean(rts,axis = 0) #Historical averages of,returns for each ticker
  marketcaps = [Data.iloc[i-1]['AAPL Market Cap'], Data.iloc[i-1]['F Market Cap'],Data.iloc[i-1]['CVS Market Cap'],Data.iloc[i-1]['CVX Market Cap']]

  marketWeights = marketcaps/np.sum(marketcaps)

  pi = Delta*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q1 = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q1),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be.
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q1])
# Black-Litterman results
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df1 = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df1['Historical Averages'] = hist_avg

  #Quadratic Programming optimization problem; https://scaron.info/doc/qpsolvers/quadratic-programming.html#primal-problem
  Q = cov
  A = np.ones((1,N))
  c = -res.reshape([N,1]) #putting negative to follow solve_qp documentation
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  lb = np.asarray([0.0001])*np.ones(N)
  ub = np.asarray([1.])*np.ones(N)
  x = solve_qp (Q, res, G, h, A, b,lb,ub, solver="quadprog")
  LastClosingPrice = Stocks_Training.iloc[-1]
  x = np.reshape(x,(N,)) #reshaping x for matrix multiplication
  AmtofShares = np.floor((x*Capital)/LastClosingPrice) #Amount of shares to purchase on last day of stock training data
  Abs_Returns = (AmtofShares * Stocks_Testing.to_numpy().reshape(-1) - AmtofShares*LastClosingPrice) #Profit/Loss
  Percent_Returns = np.sum(Abs_Returns)/Capital #Profit/Loss Percentage
  Returns.append(Percent_Returns)
  Dates.append(Stocks_Testing.index)
  print("Date: ", Stocks_Testing.index[0])
  print("Marketcaps: ",marketcaps)
  print("Weights: ", x)
  print("Amt of Shares: ", AmtofShares.to_numpy())
  print("LastClosingPrice: ", LastClosingPrice.to_numpy())
  print("NextDayPrice: ", Stocks_Testing.to_numpy())
  print("Absolute Returns: ", Abs_Returns.to_numpy)
  print("Percent Returns: ", Percent_Returns )
  print("------------------------------------------------------------------------")
d = {'Returns': Returns}
df2 = pd.DataFrame(data=d,index = np.array(Dates).reshape(-1))
df2

Date:  2022-10-20 00:00:00
Marketcaps:  [2311938383000.0, 48765481100.0, 120333888400.0, 328849048800.0]
Weights:  [1.00000000e-04 3.16890351e-01 6.82909649e-01 1.00000000e-04]
Amt of Shares:  [   0. 2612.  745.    0.]
LastClosingPrice:  [143.86  12.13  91.66 168.  ]
NextDayPrice:  [[143.39  11.77  90.99 168.96]]
Absolute Returns:  <bound method IndexOpsMixin.to_numpy of AAPL      0.00
F      -940.32
CVS    -499.15
CVX       0.00
Name: 2022-10-19 00:00:00, dtype: float64>
Percent Returns:  -0.014394699999999976
------------------------------------------------------------------------
Date:  2022-11-18 00:00:00
Marketcaps:  [2397671545000.0, 55925072900.0, 124734879000.0, 355963519900.0]
Weights:  [1.000e-04 1.000e-04 9.997e-01 1.000e-04]
Amt of Shares:  [   0.    0. 1053.    0.]
LastClosingPrice:  [150.72  13.91  94.93 184.09]
NextDayPrice:  [[151.29  13.99  97.35 182.99]]
Absolute Returns:  <bound method IndexOpsMixin.to_numpy of AAPL       0.00
F          0.00
CVS     2548.26
CVX     

,Returns
2022-10-20,-0.014395
2022-11-18,0.025483


In [ ]:



#import numpy as np
#from qpsolvers import solve_qp

#M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
#P = M.T @ M  # this is a positive definite matrix
#q = np.array([3.0, 2.0, 3.0]) @ M
#G = np.array([[1.0, 2.0, 1.0], [2.0, 0.0, 1.0], [-1.0, 2.0, -1.0]])
#h = np.array([3.0, 2.0, -2.0])
#A = np.array([1.0, 1.0, 1.0])
#b = np.array([1.0])

#x = solve_qp(P, q, G, h, A, b, solver="proxqp")
#print(f"QP solution: x = {x}")





#Quadratic Programming optimization problem; https://scaron.info/doc/qpsolvers/quadratic-programming.html#primal-problem
Q = cov
A = np.ones((1,N))
c = -res.reshape([N,1]) #putting negative to follow solve_qp documentation
b = np.array([1])
#G = np.array([[50,51,0,53]])
#h = np.asarray([25], dtype=float)
lb = np.asarray([0.0001])*np.ones(N)
ub = np.asarray([1.])*np.ones(N)
x = solve_qp (Q, res, G,h, A, b,lb,ub, solver="quadprog")
print(f"QP solution: x = {x}")

QP solution: x = [1.000e-04 1.000e-04 9.997e-01 1.000e-04]


In [ ]:
#Need to multiply market caps by 1,000,000
Data = pd.read_excel('ESG Data(Cleaned).xlsx')
# splitting data into 75% training and 25% testing
Training_Data = Data.iloc[:4500,:]
Testing_Data = Data.iloc[4500:,:]
Testing_Data.tail()

In [ ]:
#Setting the index in the dataset
Data.set_index('Dates')
Training_Data.set_index('Dates')
Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
Benchmark = Data['SPY']
Benchmark_Training = Training_Data['SPY']
Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
N = len(Stocks.columns)
LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
tickers = ['AAPL','F','CVS','CVX']

In [ ]:
Training_Data

,Dates,AAPL Market Cap,AAPL,AAPL ESG Score,F Market Cap,F,F ESG Score,CVS Market Cap,CVS,CVS ESG Score,CVX Market Cap,CVX,CVX ESG Score,SPY
0,2000-01-04,16481.8971,0.915,NaN,61149.1809,48.6339,NaN,14692.6128,18.7500,NaN,54880.2435,41.8125,NaN,139.75
1,2000-01-05,16723.0956,0.929,NaN,61375.6594,48.8140,NaN,15500.7065,19.7813,NaN,55864.6425,42.5625,NaN,140.00
2,2000-01-06,15275.9046,0.848,NaN,61451.1522,48.8740,NaN,14398.7606,18.3750,NaN,58243.6067,44.3750,NaN,137.75
3,2000-01-07,15999.5001,0.888,NaN,65980.7212,52.4765,NaN,14741.5882,18.8125,NaN,59269.0223,45.1563,NaN,145.75
4,2000-01-10,15718.1019,0.873,NaN,64772.8361,51.5159,NaN,14717.1005,18.7813,NaN,57669.3740,43.9375,NaN,146.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,2017-03-28,754452.4576,35.950,30.0,46300.5617,11.6500,58.0,80630.2444,78.6100,78.0,202883.8514,107.1700,60.0,235.32
4496,2017-03-29,756131.3504,36.030,30.0,46419.7906,11.6800,58.0,80784.0993,78.7600,78.0,204587.6442,108.0700,60.0,235.54
4497,2017-03-30,755134.5078,35.983,30.0,46419.7906,11.6800,58.0,80876.4123,78.8500,78.0,204038.6443,107.7800,60.0,236.29
4498,2017-03-31,753717.9420,35.915,30.0,46349.2896,11.6400,58.0,81309.6043,78.5000,78.0,203262.4720,107.3700,60.0,235.74


In [ ]:
m = len(Stocks_Training)
Stocks_Training

,AAPL,F,CVS,CVX
Dates,,,,
2000-01-04,0.915,48.6339,18.7500,41.8125
2000-01-05,0.929,48.8140,19.7813,42.5625
2000-01-06,0.848,48.8740,18.3750,44.3750
2000-01-07,0.888,52.4765,18.8125,45.1563
2000-01-10,0.873,51.5159,18.7813,43.9375
...,...,...,...,...
2017-03-28,35.950,11.6500,78.6100,107.1700
2017-03-29,36.030,11.6800,78.7600,108.0700
2017-03-30,35.983,11.6800,78.8500,107.7800


In [ ]:
#Returns on training data for the stocks
rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
v2 = (rts_training[1]-rts_training[2])/m
v3 = (rts_training[2]-rts_training[3])/m
v1

0.008672681825214672

In [ ]:
#Amount it will outperform the other ticker etc.
views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
vol = mgarch.mgarch()
vol.fit(rts)
nquarts = 1 #How many quarters in the future do you want to estimate the covariance
cov_nextday = vol.predict(nquarts)
cov = cov_nextday['cov']
ER = rts_Benchmark #Stocks excess returns.
ER_avg = np.mean(ER)#Historical averages of excess returns
hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
varBenchmark = np.var(rts_Benchmark)
A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
marketWeights = marketcaps/np.sum(marketcaps)

pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
df['Historical Averages'] = hist_avg
df

,BL Expected Returns,Historical Averages
AAPL,0.003571,0.001174
F,-0.000809,0.000059
CVS,-0.000377,0.000489
CVX,-0.000550,0.000339


In [ ]:
c = np.reshape(res,(N,1))
Q = cov

In [ ]:
G = np.array([[100-ESGSCORE,51,0,53]])
h = np.asarray([50], dtype=float)

NameError: ignored

In [ ]:
Q = cov
A = np.ones((1,N))
c = res
c = c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)
a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))

In [ ]:
#this is here because I don't want to mess with it

Training_Data = Data.iloc[0:4434]
Testing_Data = Data.iloc[4434:4435]
#Setting the index in the dataset
Data.set_index('Dates')
Training_Data.set_index('Dates')
Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
Benchmark = Data['SPY']
Benchmark_Training = Training_Data['SPY']
Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
N = len(Stocks.columns)
LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
tickers = ['AAPL','F','CVS','CVX']
m = len(Stocks_Training)
#Returns on training data for the stocks
rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
v2 = (rts_training[1]-rts_training[2])/m
v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
vol = mgarch.mgarch()
vol.fit(rts)
nquarts = 1 #How many quarters in the future do you want to estimate the covariance
cov_nextday = vol.predict(nquarts)
cov = cov_nextday['cov']
ER = rts_Benchmark #Stocks excess returns.
ER_avg = np.mean(ER)#Historical averages of excess returns
hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
varBenchmark = np.var(rts_Benchmark)
A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
marketWeights = marketcaps/np.sum(marketcaps)

pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
df['Historical Averages'] = hist_avg
Q = cov
A = np.ones((1,N))
c = res
c = c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)
a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))
x

In [ ]:
#range(start, stop, step)
data_for_stock_returns_dataframe = []
#data_for_weights_dataframe = {}
data_for_returns_dataframe = []
results = []
Data = pd.read_excel('ESG Data(Cleaned).xlsx')
for i in range(5885,5990,21):#start at 4434 for full and 5885 for testing
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1]


  #Setting the index in the dataset
  Data.set_index('Dates')
  Training_Data.set_index('Dates')
  Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
  Benchmark = Data['SPY']
  Benchmark_Training = Training_Data['SPY']
  Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
  Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  N = len(Stocks.columns)
  LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
  tickers = ['AAPL','F','CVS','CVX']
  m = len(Stocks_Training)
#Returns on training data for the stocks
  rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
  rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
  v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
  v2 = (rts_training[1]-rts_training[2])/m
  v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  nquarts = 1 #How many quarters in the future do you want to estimate the covariance
  cov_nextday = vol.predict(nquarts)
  cov = cov_nextday['cov']
  ER = rts_Benchmark #Stocks excess returns.
  ER_avg = np.mean(ER)#Historical averages of excess returns
  hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
  varBenchmark = np.var(rts_Benchmark)
  A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
  marketcaps = [Data.iloc[i-1]['AAPL Market Cap'], Data.iloc[i-1]['F Market Cap'],Data.iloc[i-1]['CVS Market Cap'],Data.iloc[i-1]['CVX Market Cap']]

  marketWeights = marketcaps/np.sum(marketcaps)

  pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df['Historical Averages'] = hist_avg
  Q = cov
  A = np.ones((1,N))
  c = res
  c = c.reshape([N,1])
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
  a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
  a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
  lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))



  print (x)

  z1 = (Data.iloc[i-1]['AAPL'])#price of stock at close of last training day for AAPL
  z2 = (Data.iloc[i]['AAPL'])#price of stock at close of testing day for AAPL
  f1 = (Data.iloc[i-1]['F'])#price of stock at close of last training day for F
  f2 = Data.iloc[i]['F']#price of stock at close of testing day for F
  c1 = (Data.iloc[i-1]['CVS'])#price of stock at close of last training day for CVS
  c2 = (Data.iloc[i]['CVS'])#price of stock at close of last testing day for CVS
  cv1 = (Data.iloc[i-1]['CVX'])#price of stock at close of last training day for CVX
  cv2 = Data.iloc[i]['CVX']#price of stock at close of testing day for CVX

  value_of_portfolio = ((((x[0]) *100000)/z1) *z2  + (((x[1]) *100000)/f1) *f2 + (((x[2]) *100000)/c1) *c2 + (((x[3]) *100000)/cv1) *cv2)

  rate_of_return = (value_of_portfolio -100000)/100000

  #print (value_of_portfolio)
  print (rate_of_return)

  results.append(rate_of_return)

  #print (np.mean(results))
  date_of_getting_weights = Data.iloc[i-1]['Dates'].date()

  the_date = Data.iloc[i]['Dates'].date()#changing dates column to get rid of hours, second etc.


  data_for_returns_dataframe.append([the_date ,float(rate_of_return)])#creating columns for dataframe for returns and dates
  data_for_stock_returns_dataframe.append ([date_of_getting_weights, float(x[0]), float(x[1]), float(x[2]), float(x[3])])
  #print(data_for_returns_dataframe)
  #print(data_for_weights_dataframe)



[[ 45.36567904]
 [-17.51326755]
 [ -1.37868872]
 [-25.47372278]]
[0.1958212]
[[ 61.45204748]
 [-22.43970021]
 [ -2.10333475]
 [-35.90901251]]
[-0.45183426]
[[ 33.02600532]
 [  0.96896002]
 [ -1.37765917]
 [-31.61730618]]
[-0.20769042]
[[ 35.55862193]
 [-13.12465394]
 [ -0.98918034]
 [-20.44478765]]
[-0.03680924]
[[ 22.28567234]
 [ -7.91429783]
 [ -0.43449852]
 [-12.936876  ]]
[-0.33921987]


In [ ]:



df_1 = pd.DataFrame(data_for_returns_dataframe,columns=['Dates', 'Returns'])
df_2 = pd.DataFrame(data_for_stock_returns_dataframe,columns=['Date of Trade', 'AAPL','F','CVS','CVX'] )



df_1




,Dates,Returns
0,2022-07-26,0.195821
1,2022-08-24,-0.451834
2,2022-09-22,-0.207690
3,2022-10-21,-0.036809
4,2022-11-21,-0.339220


In [ ]:
!pip install qpsolvers
import numpy as np
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
P = np.ones((len(Q),len(Stocks_Training.columns)))*views2
A = np.ones((1,N))

c = c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)






lb = 0
ub =1
x = solve_qp(P, q, G, h, A, b, solver="cvxopt")
print(f"QP solution: x = {x}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


NameError: ignored